# VGN Evaluation

Compute test metrics, plot learning curves and run simulated cutter removal experiments with a trained model.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.chdir('..')

In [ ]:
from __future__ import print_function, division

from pathlib2 import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rospy

from vgn import benchmark
from vgn.dataset import Dataset
from vgn_ros import vis

In [ ]:
class SimpleNamespace(object):
    def __init__ (self, **kwargs):
        self.__dict__.update(kwargs)

In [ ]:
rospy.init_node("vgn_evaluation", anonymous=True)

## Network

Loss and accuracy on test dataset.

## Benchmark

Metrics and failure cases of a clutter removal experiment.

In [ ]:
logdir = Path("data/experiments/meh")

Compute metrics.

In [ ]:
success_rate, percent_cleared, planning_time = benchmark.metrics(logdir)

print("Success rate   ", success_rate)
print("Percent cleared", percent_cleared)
print("Planning time  ", planning_time)

Visualize failure cases in rviz.

In [ ]:
dataset = Dataset(logdir)
failures = dataset.df[dataset.df["label"] == 0].index.tolist()
iterator = iter(failures)

In [ ]:
i = next(iterator)
dataset.draw(i, 0.05)

## Learning Curves

Loss and grasp metrics vs training epochs.

In [ ]:
run_dir = Path("data/runs/200610-2009,dataset=train,augment=True,net=conv,batch_size=32,lr=3e-04")
exp_name = "06_train_augment"
object_set = "adversarial"
epochs_to_evaluate = [5, 10]

Run clutter removal experiment for each `epochs_to_evaluate`.

In [ ]:
for epoch in epochs_to_evaluate:
    model = run_dir / ("vgn_conv_" + str(epoch) + ".pth")
    logdir = Path("data") / "experiments" / exp_name / object_set / str(epoch)    
    
    if logdir.exists():
        continue  # manually delete folder to rerun benchmark
    
    args = SimpleNamespace(
        model=model,
        logdir=logdir,
        rounds=40,
        object_set=object_set,
        object_count=5,
        seed=1,
        sim_gui=False,
        rviz=False,
    )
    benchmark.main(args)

Read train and validation losses (csvs need to be downloaded from TensorBoard).

In [ ]:
df = pd.read_csv(run_dir / "train" / "loss.csv")
train_epochs = df["Step"].to_numpy()
train_loss = df["Value"].to_numpy()

df = pd.read_csv(run_dir / "validation" / "loss.csv")
val_epochs = df["Step"].to_numpy()
val_loss = df["Value"].to_numpy()

Read the benchmark metrics.

In [ ]:
root = Path("data") / "experiments" / exp_name / object_set
success_rates = []
for epoch in epochs_to_evaluate:
    log_dir = root / str(epoch)
    success_rate, _, _ = benchmark.metrics(log_dir)
    success_rates.append(success_rate)

Create plot.

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

l1, = ax1.plot(train_epochs, train_loss, color="C0")
l2,= ax1.plot(val_epochs, val_loss, color="C1")
l3, = ax2.plot(epochs_to_evaluate, success_rates, color="C2")

ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax2.set_ylabel("%")

ax1.legend([l1, l2], ["train", "validation"], loc="lower left")
ax2.legend([l3], ["success rate"], loc="upper right")

fig.suptitle(exp_name)
fig_path = Path.home() / "Desktop" / (exp_name + ".png")
plt.savefig(str(fig_path))
plt.show()